In [22]:
# Test
workflows = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}"""

ratings="""{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

In [54]:
# Real
workflows = """REDACTED"""

ratings = """REDACTED"""

In [51]:
workflow_map = {}

for workflow in workflows.splitlines():
    name = workflow.split("{")[0]
    rules = workflow.split("{")[1][:-1]
    rule_vec = []
    for rule in rules.split(","):
        # rules are tuple (prop, cond, amnt, destination)
        if rule.count(":") == 0:
            rule_vec.append(("x", ">", 0, rule))
        else:
            cond = "<" if rule.count("<") > 0 else ">" 
            dest = rule.split(":")[-1]
            prop = rule.split(":")[0].split(cond)[0]
            amnt = int(rule.split(":")[0].split(cond)[1])
            rule_vec.append((prop, cond, amnt, dest))
    workflow_map[name] = rule_vec

ratings_list = []
for rating in ratings.splitlines():
    rat = {}
    for val in rating[1:-1].split(","):
        rat[val.split("=")[0]] = int(val.split("=")[1])
    ratings_list.append(rat)

sum1 = 0
current = workflow_map["in"]

for rating in ratings_list:
    done = False
    new_rule = workflow_map["in"]
    while not done:
        for rule in current:
            if rule[1] == "<":
                if rating[rule[0]] < rule[2]:
                    if rule[3] == "A":
                        sum1 += sum([x for x in rating.values()])
                        done = True
                        break
                    elif rule[3] == "R":
                        done = True
                        break
                    else:
                        new_rule = workflow_map[rule[3]]
                        break
            elif rule[1] == ">":
                if rating[rule[0]] > rule[2]:
                    if rule[3] == "A":
                        sum1 += sum([x for x in rating.values()])
                        done = True
                        break
                    elif rule[3] == "R":
                        done = True
                        break
                    else:
                        new_rule = workflow_map[rule[3]]
                        break
        current = new_rule
    current = new_rule = workflow_map["in"]

print("Part 1: ", sum1)
    


Part 1:  332145


In [53]:
### IMPORTANT, MUST RERUN ABOVE CELL IF CHANGING INPUTS

from copy import deepcopy
# Part 2
# ranges are tuple ({x : (x_min, x_max), m : (m_min, m_max), a : (a_min, a_max), s : (s_min, s_max)}, current_rule
ranges_to_check = [({"x" : (1, 4000), "m" : (1, 4000), "a" : (1, 4000), "s" : (1, 4000)}, "in")]
accepted_ranges = []

steps = 0

while len(ranges_to_check) > 0:
    steps += 1
    print(steps)
    current = ranges_to_check.pop()
    ranges = current[0]
    workflow = workflow_map[current[1]]
    # Not super efficient but let's just go through a rule until we hit something that splits the range or the end
    # rules are tuple (prop, cond, amnt, destination)
    done = False
    for rule in workflow:
        if ranges[rule[0]][0] < rule[2] and ranges[rule[0]][1] > rule[2]:
            
            if rule[1] == "<":
                new_range1 = deepcopy(ranges)
                new_range1[rule[0]] = (new_range1[rule[0]][0], rule[2] - 1)
                new_range2 = deepcopy(ranges)
                new_range2[rule[0]] = (rule[2], new_range2[rule[0]][1])
                ranges_to_check.append((new_range2, current[1]))
                if rule[3] == "A":
                    accepted_ranges.append(new_range1)
                    # if len(accepted_ranges) == 2:
                    #     print(current, rule)
                    #     print(accepted_ranges)
                elif rule[3] != "R":
                    ranges_to_check.append((new_range1, rule[3]))
            if rule[1] == ">":
                new_range1 = deepcopy(ranges)
                new_range1[rule[0]] = (new_range1[rule[0]][0], rule[2])
                new_range2 = deepcopy(ranges)
                new_range2[rule[0]] = (rule[2] + 1, new_range2[rule[0]][1])
                ranges_to_check.append((new_range1, current[1]))
                if rule[3] == "A":
                    accepted_ranges.append(new_range2)
                    # if len(accepted_ranges) == 2:
                    #     print(current, rule)
                    #     print(accepted_ranges)
                elif rule[3] != "R":
                    ranges_to_check.append((new_range2, rule[3]))
            done = True
            break
        else:
            if rule[1] == "<":
                if ranges[rule[0]][1] < rule[2]:
                    done = True
                    if rule[3] == "A":
                        accepted_ranges.append(ranges)
                        # if len(accepted_ranges) == 2:
                        #     print(current, rule)
                        #     print(accepted_ranges)
                    elif rule[3] != "R":
                        ranges_to_check.append((ranges, rule[3]))
                    break
                else:
                    continue
            if rule[1] == ">":
                if ranges[rule[0]][0] > rule[2]:
                    done = True
                    if rule[3] == "A":
                        accepted_ranges.append(ranges)
                        # if len(accepted_ranges) == 2:
                        #     print(current, rule)
                        #     print(accepted_ranges)
                    elif rule[3] != "R":
                        ranges_to_check.append((ranges, rule[3]))
                    break
                else:
                    continue
        
print(accepted_ranges)

sum2 = 0
for ranges in accepted_ranges:
    total = 1
    for val_pair in ranges.values():
        total *= val_pair[1] - val_pair[0] + 1
    sum2 += total

print("Part 2: ", sum2)

167409079868000
167409079868000

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


167409079868000